# CLASS/POLICY BASED ROUTING

## RUN EXPERIMENT

## SETUP FABRIC CONFIGURATION

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config()

# set appropriate permission on your keys
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,vsunda21_0000056251
Bastion Private Key File,/home/fabric/work/fabric_config/bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
import json
import traceback
import os

SLICENAME="Class/Policy-based-Routing_" + os.getenv('NB_USER')


try:
    slice = fablib.get_slice(SLICENAME)
    print("You already have a slice named %s." % SLICENAME)
    print(slice)
except:
    slice = fablib.new_slice(name=SLICENAME)
    print("You will need to create a %s slice." % SLICENAME)

You will need to create a Class/Policy-based-Routing_vsunda21 slice.


## SITE SELECTION 

In [3]:
exp_requires = {'core': 4*3, 'nic': 4}
while True:
    site_name = fablib.get_random_site()
    site_name = 'DALL'
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break

fablib.show_site(site_name)

Name,DALL
Address,"1950 N Stemmons Fwy,Dallas, TX 75207"
Location,"(32.8002714, -96.8198113)"
Hosts,3
CPUs,6
Cores Available,120
Cores Capacity,192
Cores Allocated,72
RAM Available,1296
RAM Capacity,1536
RAM Allocated,240


'<pandas.io.formats.style.Styler object at 0x7f4a902bc370>'

In [4]:
# this cell sets up hosts and routers
node_names = ["Source", "node1", "node2", "node3", "Receiver"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=4, disk=100, image='default_ubuntu_20')

In [5]:
# this cell sets up the network links
nets = [
    {"name": "net0",  "nodes": ["Source", "node1"]},
    {"name": "net1",  "nodes": ["node1", "node2"]},
    {"name": "net2",  "nodes": ["node1", "node3"]},
    {"name": "net3",  "nodes": ["node2", "node3"]},
    {"name": "net4",  "nodes": ["node3", "Receiver"]}
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

In [6]:
slice.submit()


Retry: 7, Time: 294 sec


ID,10623397-c50c-4571-82d0-c36517fd93f8
Name,Class/Policy-based-Routing_vsunda21
Lease Expiration (UTC),2023-05-03 04:26:59 +0000
Lease Start (UTC),2023-05-02 04:27:00 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
f4b17264-43e1-42cb-bf9a-954360ff466e,Source,4,8,100,default_ubuntu_20,qcow2,dall-w3.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe09:8b1a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe09:8b1a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7bafcf4f-664b-4f70-8305-be591f6898f4,node1,4,8,100,default_ubuntu_20,qcow2,dall-w3.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe87:2f7a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe87:2f7a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6b26b064-f830-4197-ab14-d23636bc400e,node2,4,8,100,default_ubuntu_20,qcow2,dall-w3.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:feef:c588,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feef:c588,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8cb859f0-a952-44cf-a938-5b95ef81779a,node3,4,8,100,default_ubuntu_20,qcow2,dall-w3.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fedb:8c39,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fedb:8c39,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5d512115-f8e0-404c-b43a-124732b333b1,Receiver,4,8,100,default_ubuntu_20,qcow2,dall-w3.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe01:9a48,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe01:9a48,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
9077eb02-4610-493a-ab8b-f516df390c11,net0,L2,L2Bridge,DALL,None,None,Active,
ba37dfeb-2abc-4acc-a445-daf6b7b047d7,net1,L2,L2Bridge,DALL,None,None,Active,
bafd1218-1f91-456e-b9e0-a5634a563411,net2,L2,L2Bridge,DALL,None,None,Active,
40b9458d-b508-4046-9b8d-1c3e18bede8a,net3,L2,L2Bridge,DALL,None,None,Active,
f2f53e94-9cc8-43a3-805c-a937acda9877,net4,L2,L2Bridge,DALL,None,None,Active,



Time to stable 294 seconds
Running post_boot_config ... Time to post boot config 376 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
Source-net0-p1,Source,net0,100,None,06:A5:E9:90:73:2D,ens7,ens7
node1-net2-p1,node1,net2,100,None,12:B6:99:D7:6A:85,ens7,ens7
node1-net0-p1,node1,net0,100,None,16:71:02:13:00:E7,ens9,ens9
node1-net1-p1,node1,net1,100,None,16:6E:E3:AF:BC:D2,ens8,ens8
node2-net1-p1,node2,net1,100,None,0A:E9:54:EE:5A:01,ens8,ens8
node2-net3-p1,node2,net3,100,None,0A:62:E7:BC:5A:BC,ens7,ens7
node3-net3-p1,node3,net3,100,None,12:8D:D3:3F:B8:6F,ens9,ens9
node3-net2-p1,node3,net2,100,None,0E:C8:50:3C:20:E5,ens7,ens7
node3-net4-p1,node3,net4,100,None,12:7B:21:C1:33:9A,ens8,ens8
Receiver-net4-p1,Receiver,net4,100,None,16:8D:92:B5:DC:B3,ens7,ens7



Time to print interfaces 400 seconds


'10623397-c50c-4571-82d0-c36517fd93f8'

In [7]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

In [8]:
for node in slice.get_nodes():
    print(f"{node}")

-----------------  --------------------------------------------------------------------------------------------------------------------------------------------
ID                 f4b17264-43e1-42cb-bf9a-954360ff466e
Name               Source
Cores              4
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               dall-w3.fabric-testbed.net
Site               DALL
Management IP      2001:400:a100:3000:f816:3eff:fe09:8b1a
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe09:8b1a
-----------------  --------------------------------------------------------------------------------------------------------------------------------------------
-----------------  ----------------------------------------------------------------------------------------------------------------------------

In [9]:
# variables specific to this slice
Source_IP = str(slice.get_node("Source").get_management_ip())
Source_USER =  str(slice.get_node("Source").get_username())
Source_IFACE_V = slice.get_node("Source").get_interfaces()[0].get_os_interface()

Receiver_IP = str(slice.get_node("Receiver").get_management_ip())
Receiver_USER =  str(slice.get_node("Receiver").get_username())
Receiver_IFACE_1 = slice.get_node("Receiver").get_interfaces()[0].get_os_interface()

node1_IP = str(slice.get_node("node1").get_management_ip())
node1_USER =  str(slice.get_node("node1").get_username())
node1_IFACE_0 = slice.get_node("node1").get_component("net0").get_interfaces()[0].get_os_interface()
node1_IFACE_1 = slice.get_node("node1").get_component("net1").get_interfaces()[0].get_os_interface()
node1_IFACE_2 = slice.get_node("node1").get_component("net2").get_interfaces()[0].get_os_interface()

node2_IP = str(slice.get_node("node2").get_management_ip())
node2_USER =  str(slice.get_node("node2").get_username())
node2_IFACE_0 = slice.get_node("node2").get_component("net1").get_interfaces()[0].get_os_interface()
node2_IFACE_1 = slice.get_node("node2").get_component("net3").get_interfaces()[0].get_os_interface()

node3_IP = str(slice.get_node("node3").get_management_ip())
node3_USER =  str(slice.get_node("node3").get_username())
node3_IFACE_0 = slice.get_node("node3").get_component("net2").get_interfaces()[0].get_os_interface()
node3_IFACE_1 = slice.get_node("node3").get_component("net3").get_interfaces()[0].get_os_interface()
node3_IFACE_2 = slice.get_node("node3").get_component("net4").get_interfaces()[0].get_os_interface()

FABRIC_SLICE_PRIVATE_KEY_FILE = fablib.get_default_slice_private_key_file
FABRIC_BASTION_USERNAME = fablib.get_bastion_username()
FABRIC_BASTION_HOST = fablib.get_bastion_public_addr()

SSH_CMD_Source=slice.get_node('Source').get_ssh_command()
SSH_CMD_Receiver=slice.get_node('Receiver').get_ssh_command()
SSH_CMD_node1=slice.get_node('node1').get_ssh_command()
SSH_CMD_node2=slice.get_node('node2').get_ssh_command()
SSH_CMD_node3=slice.get_node('node3').get_ssh_command()

## ONE TIME SETUP

In [18]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf = {
    "Source-net0-p1":   {"addr": "192.168.0.1", "subnet": "192.168.0.0/24"},
    "node1-net0-p1":  {"addr": "192.168.0.2", "subnet": "192.168.0.0/24"},
    "node1-net1-p1":  {"addr": "192.168.1.1", "subnet": "192.168.1.0/24"},
    "node1-net2-p1":  {"addr": "192.168.2.1", "subnet": "192.168.2.0/24"},
    "node2-net1-p1":  {"addr": "192.168.1.2", "subnet": "192.168.1.0/24"},
    "node2-net3-p1":  {"addr": "192.168.3.1", "subnet": "192.168.3.0/24"},
    "node3-net2-p1":  {"addr": "192.168.2.2", "subnet": "192.168.2.0/24"},
    "node3-net3-p1":  {"addr": "192.168.3.2", "subnet": "192.168.3.0/24"},
    "node3-net4-p1":  {"addr": "192.168.4.1", "subnet": "192.168.4.0/24"},
    "Receiver-net4-p1":  {"addr": "192.168.4.2", "subnet": "192.168.4.0/24"}
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))

In [19]:
rt_conf = [
    {"name": "Source",   "addr": "192.168.2.0/24", "gw": "192.168.0.2"},
    {"name": "Source",   "addr": "192.168.4.0/24", "gw": "192.168.0.2"},
    {"name": "node1",   "addr": "192.168.4.0/24", "gw": "192.168.2.2"},
    {"name": "node1",   "addr": "192.168.4.0/24", "gw": "192.168.1.2"},
    {"name": "node2",   "addr": "192.168.4.0/24", "gw": "192.168.3.2"},
    {"name": "node2",   "addr": "192.168.0.0/24", "gw": "192.168.1.1"},
    {"name": "node3",   "addr": "192.168.0.0/24", "gw": "192.168.2.1"},
    {"name": "Receiver",   "addr": "192.168.0.0/24", "gw": "192.168.4.1"},
    {"name": "Receiver",   "addr": "192.168.2.0/24", "gw": "192.168.4.1"}
]
for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])


In [20]:
for n in ['node1','node2','node3']:
    slice.get_node(name=n).execute("sudo sysctl -w net.ipv4.ip_forward=1")

net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1


In [21]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [22]:
for node in ["Source", "Receiver", "node1","node2","node3"]:
    slice.get_node(node).execute("sudo sed -i '/nameserver/d' /etc/resolv.conf")
    slice.get_node(node).execute("echo nameserver 2a00:1098:2c::1 | sudo tee -a /etc/resolv.conf")
    slice.get_node(node).execute("echo nameserver 2a01:4f8:c2c:123f::1 | sudo tee -a /etc/resolv.conf")
    slice.get_node(node).execute("echo nameserver 2a00:1098:2b::1 | sudo tee -a /etc/resolv.conf")
    slice.get_node(node).execute('echo "127.0.0.1 $(hostname -s)" | sudo tee -a /etc/hosts')

nameserver 2a00:1098:2c::1
nameserver 2a01:4f8:c2c:123f::1
nameserver 2a00:1098:2b::1
127.0.0.1 f4b17264-43e1-42cb-bf9a-954360ff466e-source
nameserver 2a00:1098:2c::1
nameserver 2a01:4f8:c2c:123f::1
nameserver 2a00:1098:2b::1
127.0.0.1 5d512115-f8e0-404c-b43a-124732b333b1-receiver
nameserver 2a00:1098:2c::1
nameserver 2a01:4f8:c2c:123f::1
nameserver 2a00:1098:2b::1
127.0.0.1 7bafcf4f-664b-4f70-8305-be591f6898f4-node1
nameserver 2a00:1098:2c::1
nameserver 2a01:4f8:c2c:123f::1
nameserver 2a00:1098:2b::1
127.0.0.1 6b26b064-f830-4197-ab14-d23636bc400e-node2
nameserver 2a00:1098:2c::1
nameserver 2a01:4f8:c2c:123f::1
nameserver 2a00:1098:2b::1
127.0.0.1 8cb859f0-a952-44cf-a938-5b95ef81779a-node3


In [23]:
for n in ['Source', 'node1', 'node2', 'node3', 'Receiver']:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install net-tools iperf3 moreutils", quiet=True)

In [24]:
for n in ['Source', 'node1', 'node2', 'node3', 'Receiver']:
    slice.get_node(name=n).execute("touch ~/.hushlogin", quiet=True)

In [25]:
%%bash -s "$SSH_CMD_Source"
$1 << EOF
##############################################

ping -c 5 192.168.4.2

##############################################
exit
EOF

PING 192.168.4.2 (192.168.4.2) 56(84) bytes of data.
64 bytes from 192.168.4.2: icmp_seq=1 ttl=62 time=0.601 ms
64 bytes from 192.168.4.2: icmp_seq=2 ttl=62 time=0.317 ms
64 bytes from 192.168.4.2: icmp_seq=3 ttl=62 time=0.215 ms
64 bytes from 192.168.4.2: icmp_seq=4 ttl=62 time=0.292 ms
64 bytes from 192.168.4.2: icmp_seq=5 ttl=62 time=0.232 ms

--- 192.168.4.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4085ms
rtt min/avg/max/mdev = 0.215/0.331/0.601/0.139 ms


Pseudo-terminal will not be allocated because stdin is not a terminal.


In [26]:
%%bash -s "$SSH_CMD_Receiver"
$1 << EOF
##############################################
# Install Curl
sudo apt update
sudo apt install curl -y

# Install Apache (HTTP server)
sudo apt install apache2 -y

# Install pure-ftpd(FTP server)
sudo apt install pure-ftpd -y

touch vignesh

echo "Hello, world!" > vignesh

echo "Installation completed successfully."
##############################################
exit
EOF

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
204 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libcurl4
The following packages will be upgraded:
  curl libcurl4
2 upgraded, 0 newly installed, 0 to remove and 202 not upgraded.
Need to get 397 kB of archives.
After this operation, 10.2 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 curl amd64 7.68.0-1ubuntu2.18 [161 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 lib

Pseudo-terminal will not be allocated because stdin is not a terminal.




debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs she

# Experiment1 (Commands to successfully complete Experiment 1)

#### Configure password at ftp server (Receiver)
```bash
sudo passwd ubuntu
```
### Run these commands on the Source terminal to generate http and ftp traffic
```bash
# to view the timetaken and send multiple requests
function time_taken {
    echo "$1 took $(($(date +%s%N) - $start_time)) nanoseconds"
}

for i in {1..100}; do
    # HTTP request
    start_time=$(date +%s%N)
    curl -I http://192.168.4.2 > /dev/null 2>&1
    time_taken "HTTP request"

    # FTP request
    start_time=$(date +%s%N)
    curl -u ubuntu:ubuntu ftp://192.168.4.2/vignesh > /dev/null 2>&1
    time_taken "FTP request"
done

# Total time taken
echo "Total time taken: $(($(date +%s%N) - $start_time)) nanoseconds"

```

```bash
for i in {1..100}; do curl -I http://192.168.4.2 & curl -u ubuntu:ubuntu ftp://192.168.4.2/vignesh & done ### for multiple requests
```
```bash
# single request
curl -I http://192.168.4.2 
curl -u ubuntu:ubuntu ftp://192.168.4.2/vignesh #ubuntu is the username and password
```


### Run these commands on the node1 terminal (separate terminals) to capture the http and ftp packets (Note: please change the correct interface(ens8,9,7) according to the slice (interface changes everytime once you delete and run)
```bash
sudo tcpdump -i ens8 -n host 192.168.4.2 and \(tcp port 80 or tcp port 21\)
```
Note: Replace the required interfaces to view the traffic; Running this at 2 interfaces in node1 is sufficient to see which path the traffic takes

# Experiment 2 (commands to SEPARATE HTTP and FTP traffic)

### Run these commands on the terminal of the node1 to send the ftp traffic through different path to destination
```bash
sudo sh -c 'echo "10 ftp" >> /etc/iproute2/rt_tables'
sudo ip rule add fwmark 1 table 10 priority 100
sudo iptables -t mangle -A PREROUTING -p tcp --dport 21 -j MARK --set-mark 1
sudo ip route add 192.168.4.2 via 192.168.1.2 dev ens7 table 10
```
### Run these commands on the terminal of the node2 to send the ftp traffic through different path to destination
```bash
sudo sh -c 'echo "10 ftp" >> /etc/iproute2/rt_tables'
sudo ip rule add fwmark 1 table 10 priority 100
sudo iptables -t mangle -A PREROUTING -p tcp --dport 21 -j MARK --set-mark 1
sudo ip route add 192.168.4.2 via 192.168.3.2 dev ens7 table 10
```
### Run these commands on the terminal of the node3 to send the ftp traffic through different path to source
```bash
sudo sh -c 'echo "10 ftp" >> /etc/iproute2/rt_tables'
sudo ip rule add fwmark 1 table 10 priority 100
sudo iptables -t mangle -A PREROUTING -p tcp --sport 21 -j MARK --set-mark 1
sudo ip route add 192.168.0.1 via 192.168.3.1 dev ens9 table 10
```
### Run these commands on the terminal of the node2 to send the ftp traffic through different path to source
```bash
sudo sh -c 'echo "20 ftp" >> /etc/iproute2/rt_tables'
sudo ip rule add fwmark 1 table 20 priority 100
sudo iptables -t mangle -A PREROUTING -p tcp --sport 21 -j MARK --set-mark 1
sudo ip route add 192.168.0.1 via 192.168.1.1 dev ens8 table 20
```
Note: 1. Run the tcpdump commands and curl command done in experiment 1 to view and generate traffic
      2. Replace the interface ens 7/8/9 before running the commands on the terminals

### DELETE A SLICE

In [27]:
fablib.delete_slice(SLICENAME)